In [2]:
:dep svg = { version = "*" }
:dep retiles = { path = "." }
:dep based = {version = "1.0"}

In [3]:
use retiles::{Direction, Glue, NormalisedEdgePosition, RestrictedDirection, TileAssembly, TileSet};

In [4]:
use svg::node::element::path::Data;
use svg::node::element::Path;
use svg::Document;

In [5]:
fn valued_edge_to_svg(edge: (&NormalisedEdgePosition, &Glue), scale: f32) -> Vec<Path> {
    fn restricted_direction_to_vec(d: RestrictedDirection, scale: f32) -> (f32, f32) {
        match d {
            RestrictedDirection::LEFT => (-1.0 * scale, 0.0),
            RestrictedDirection::UP => (0.0, -1.0 * scale),
        }
    }

    fn glue_to_color(g: &Glue) -> String {
        match g {
            0 => "#E6C58F".into(),
            1 => "#0AC52E".into(),
            2 => "#E37998".into(),
            _ => "#3FB3F3".into(),
        }
    }

    fn replace_0_with(t: (f32, f32), r: f32) -> (f32, f32) {
        match t {
            (0.0, 0.0) => (r, r),
            (0.0, _) => (r, t.1),
            (_, 0.0) => (t.0, r),
            t => (t.0, t.1),
        }
    }

    let starting_point = (scale * (edge.0 .0 .0 as f32), -1.0 * scale * (edge.0 .0 .1 as f32));
    let vec = restricted_direction_to_vec(edge.0 .1, scale);
    let replace_vec_1 = replace_0_with(vec, scale);
    let replace_vec_2 = replace_0_with(vec, -1.0 * scale);
    let half_vec_1 = (
        (replace_vec_1.0 as f32) * 0.5,
        (replace_vec_1.1 as f32) * 0.5,
    );
    let half_vec_2 = (
        (replace_vec_2.0 as f32) * 0.5,
        (replace_vec_2.1 as f32) * 0.5,
    );

    let svg_edge = Data::new()
        .move_to(starting_point.clone())
        .line_by(vec)
        .close();

    let svg_triangle_1 = Data::new()
        .move_to(starting_point)
        .line_by(half_vec_1)
        .line_by(half_vec_2)
        .close();

    let svg_triangle_2 = Data::new()
        .move_to(starting_point)
        .line_by(half_vec_2)
        .line_by(half_vec_1)
        .close();

    let stroke_width_multiplier = 0.02;

    let path_edge = Path::new()
        .set("stroke", "gray")
        .set("stroke-width", (scale as f32) * stroke_width_multiplier)
        .set("d", svg_edge);

    let path_triangle_1 = Path::new()
        .set("stroke", "gray")
        .set("fill", glue_to_color(edge.1))
        .set("stroke-width", (scale as f32) * stroke_width_multiplier)
        .set("d", svg_triangle_1);

    let path_triangle_2 = Path::new()
        .set("stroke", "gray")
        .set("fill", glue_to_color(edge.1))
        .set("stroke-width", (scale as f32) * stroke_width_multiplier)
        .set("d", svg_triangle_2);

    [path_edge, path_triangle_1, path_triangle_2].to_vec()
}

In [6]:
struct SVGString {
    svg: String,
}
use std::fmt::Debug;
impl SVGString {
    pub fn evcxr_display(&self) {
        let mut html = String::new();
        
        html.push_str("<svg>");
        
        html.push_str(&self.svg);
        
        html.push_str("</svg>");
        println!("EVCXR_BEGIN_CONTENT text/html\n{}\nEVCXR_END_CONTENT", html);
    }
}

In [7]:
pub struct List<T>(Vec<T>);
impl List<SVGString> {
    pub fn evcxr_display(&self) {
        let mut html = String::new();
        for item in &self.0 {
            html.push_str("<svg>");
            html.push_str(&item.svg);
            html.push_str("</svg>");
        }
        println!("EVCXR_BEGIN_CONTENT text/html\n{}\nEVCXR_END_CONTENT", html);
    }
}

In [8]:
fn ta_box(ta: &TileAssembly) -> (i32, i32, i32, i32) {
    let mut min_x: i32 = 0;
    let mut max_x: i32 = 0;
    let mut min_y: i32 = 0;
    let mut max_y: i32 = 0;

    for edge in ta.edges.keys() {
        if edge.0.0 < min_x {
            min_x = edge.0.0;
        }
        if edge.0.0 > max_x {
            max_x = edge.0.0;
        }
        if edge.0.1 < min_y {
            min_y = edge.0.1;
        }
        if edge.0.1 > max_y {
            max_y = edge.0.1;
        }
    }

    (min_x, max_x, min_y, max_y)
}

In [9]:
fn ta_to_svg(ta: &TileAssembly, scale: f32) -> SVGString {
    let ta_box = ta_box(&ta);
    let mut document = Document::new().set("viewBox", (ta_box.0, -1*ta_box.3, ta_box.1-ta_box.0, ta_box.3-ta_box.2));
    // println!("ta box: {:?}", ta_box);
    // println!("viewBox: {:?}", (ta_box.0, -1*ta_box.3, ta_box.1-ta_box.0, ta_box.3-ta_box.2));
    let mut k = 0;
    for edge in ta.edges.iter() {
        let vec_path = valued_edge_to_svg(edge, scale);

        for path in vec_path {
            document = document.add(path);
        }

        k += 1;
    }

    SVGString {svg: document.to_string()}
}

In [10]:
let mut hydra_ta = TileAssembly::new(TileSet::get_collatz_tileset());
hydra_ta.add_edge(&((0, 0), Direction::LEFT), 1);
hydra_ta.add_edge(&((-1, 0), Direction::LEFT), 1);

let N = 5;

for i in 1..=N {
    hydra_ta.add_edge(&((-1 - i, 0), Direction::LEFT), 0);
}

let mut curr_position = (0, 0);

let mut k = 0;
while k <= N {
    let curr_glue = hydra_ta
        .edges
        .get(&(curr_position, RestrictedDirection::LEFT))
        .unwrap();

    curr_position = (curr_position.0 - 1, curr_position.1 - 1);
    hydra_ta.add_edge(&(curr_position, Direction::UP), *curr_glue);
    hydra_ta.solve();

    k += 1;
}

()

In [11]:
ta_to_svg(&TileAssembly::assembly_from_Collatz_parity_vector(vec![0,1,1,0,0,0,0,1,1]), 1.0)

In [12]:
use based::{Base, NumeralSystem};

fn top_left_rect_corner(x: u32, 
                        width: u32, 
                        height: u32, 
                        ensurePositiveCycle: bool, verbose: bool) -> Option<TileAssembly> {
    let base2: Base = "01".parse().unwrap();
    let base3: Base = "012".parse().unwrap();
    let x_base2 = base2.encode(x as usize).unwrap();
    let x_base3 = base3.encode(x as usize).unwrap();

    if verbose {
        println!("x_base2: {:?}", x_base2);
        println!("x_base3: {:?}", x_base3);
    }

    if ensurePositiveCycle && 2_u32.pow(width) <= 3_u32.pow(height) {
        return None;
    }

    if (width as usize) < x_base2.len() || (height as usize) < x_base3.len() {
        return None;
    }

    let mut ta = TileAssembly::new(TileSet::get_collatz_tileset());

    for i in 0..x_base2.len() {
        let curr_glue = (x_base2.chars().nth(i).unwrap() as u32 - '0' as u32) as u8;
        let x_pos = (width as usize) - x_base2.len()+i;
        ta.add_edge(&((x_pos as i32, 0 as i32), Direction::LEFT), curr_glue);
    }

    for i in 0..(width as usize-x_base2.len()) {
        ta.add_edge(&((i as i32, 0 as i32), Direction::LEFT), 0);
    }

    for i in 0..x_base3.len() {
        let curr_glue = (x_base3.chars().nth(i).unwrap() as u32 - '0' as u32) as u8;
        let y_pos = (height as usize) - x_base3.len()+i+1;
        ta.add_edge(&((-1 as i32, -1*y_pos as i32), Direction::UP), curr_glue);
        
    }

    for i in 0..(height as usize-x_base3.len()) {
        ta.add_edge(&((-1 as i32, (-1*i as i32)-1 as i32), Direction::UP), 0);
    }

    return Some(ta);
}

In [13]:
let top_left_rect_corner_ta = top_left_rect_corner(45,10,5,true, true).unwrap();
ta_to_svg(&top_left_rect_corner_ta, 1.0)

x_base2: "101101"
x_base3: "1200"


In [14]:
let solutions = top_left_rect_corner_ta.solve_non_det(2);
solutions.len()

23

In [15]:
let all_SVG = solutions.iter().map(|sol| ta_to_svg(sol, 0.5)).collect::<Vec<SVGString>>();
List(all_SVG).evcxr_display()

()

In [20]:
for x in 0..=500 {
    let top_left_rect_corner_ta = top_left_rect_corner(x,15,7,true, false).unwrap();
    let solutions = top_left_rect_corner_ta.solve_non_det(2);
    println!("{} {}", x, solutions.len());
}
// let all_SVG = solutions.iter().map(|sol| ta_to_svg(sol, 1.0)).collect::<Vec<SVGString>>();
// List(all_SVG).evcxr_display()

0 1429850
1 483989
2 427529
3 185120
4 124473
5 126897
6 109477
7 89449
8 99771
9 50531
10 36015
11 43919
12 35230
13 25277
14 28412
15 26255
16 23016
17 26231
18 22665
19 20238
20 20667
21 17770
22 15282
23 16976
24 18322
25 16372
26 18791
27 10398
28 8186
29 10034
30 7652
31 5629
32 7181
33 8114
34 6521
35 8016
36 6933
37 5207
38 6776
39 5597
40 3887
41 4478
42 4601
43 4070
44 4758
45 4170
46 3637
47 3904
48 3715
49 3278
50 3834
51 4104
52 3658
53 4268
54 3793
55 3305
56 3609
57 3488
58 3054
59 3374
60 3217
61 2792
62 3113
63 2756
64 2407
65 2736
66 2492
67 2150
68 2381
69 2532
70 2217
71 2528
72 2699
73 2411
74 2677
75 2491
76 2217
77 2503
78 2748
79 2482
80 2812
81 1507
82 1215
83 1455
84 1221
85 976
86 1225
87 1412
88 1155
89 1419
90 1112
91 813
92 1033
93 838
94 608
95 808
96 960
97 774
98 988
99 1108
100 921
101 1085
102 937
103 753
104 953
105 1102
106 919
107 1128
108 1029
109 800
110 1003
111 831
112 614
113 817
114 979
115 792
116 995
117 867
118 641
119 850
120 677
121 450


()

In [23]:
top_left_rect_corner(x,15,7,true, true).unwrap();

x_base2: "111101111"
x_base3: "200100"


In [22]:
ta_to_svg(&top_left_rect_corner_ta, 1.0)

In [18]:
let x = 495;
let top_left_rect_corner_ta = top_left_rect_corner(x,15,7,true, false).unwrap();
let solutions = top_left_rect_corner_ta.solve_non_det(2);
println!("{} {}", x, solutions.len());
let all_SVG = solutions.iter().map(|sol| ta_to_svg(sol, 0.8)).collect::<Vec<SVGString>>();
List(all_SVG).evcxr_display()

495 35


()

In [ ]:
let x = 0;
let top_left_rect_corner_ta = top_left_rect_corner(x,15,7,true, false).unwrap();
let solutions = top_left_rect_corner_ta.solve_non_det(2);
println!("{} {}", x, solutions.len());
let all_SVG = solutions.iter().map(|sol| ta_to_svg(sol, 0.8)).collect::<Vec<SVGString>>();
List(all_SVG).evcxr_display()